In [1]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
import datetime

/Users/marielwiechers/my_env/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
%run ml_helpers.ipynb
%run ml.ipynb

         Unnamed: 0  Beat  Year  Month   Watch  PRCP  SNOW  TMAX  TMIN  \
1301129     1301129   723  2019     11  Second  0.00   0.0    42    29   
1211540     1211540   511  2019      7   Third  0.32   0.0    91    74   
1129071     1129071  2413  2019      3   First  0.00   0.0    40    24   
1329418     1329418   632  2019     12  Second  0.16   0.0    51    30   
1131389     1131389   613  2019      4  Second  0.02   0.0    59    37   
...             ...   ...   ...    ...     ...   ...   ...   ...   ...   
1124462     1124462  1132  2019      3  Second  0.11   0.0    51    36   
1180521     1180521  1812  2019      6   Third  0.00   0.0    79    60   
1308846     1308846   624  2019     11  Second  0.12   0.0    57    34   
1316735     1316735  1221  2019     12   First  0.02   0.0    49    25   
1073564     1073564   631  2019      1   First  0.00   0.0    50    32   

         count_l_stops  ...  count_restaurants  count_bars  count_daycares  \
1301129            1.0  ...      

In [3]:
data = pd.read_csv("../intermediate_data/high_crime_labeled.csv")
data = data.drop(columns=["beat", "beat_num", "Crimes", "Arrest", "Domestic", "Serious"])
data = convert_to_categorical(data, ["district", "sector",
                                    "Month",
                                     "Watch", "Beat"])
data.drop(columns = ["Unnamed: 0"], inplace=True)

In [4]:
data.head()

,district,sector,Year,Month,Watch,Beat,TMAX,TMIN,PRCP,SNOW,...,count_restaurants,count_bars,count_daycares,count_entertainment,count_businesses,road_distance_ft,TOTAL POPULATION,dist_to_police,dist_to_hospital,high_crime_geog_lag
0,17,1,2015,1,Third,1713,32.469987,20.138549,1.55,11.3,...,63.0,9.0,6.0,1.0,92.0,169276.405792,13283.675264,5454.06889,3258.006066,0.155378
1,17,1,2015,1,Second,1713,32.469987,20.138549,1.55,11.3,...,63.0,9.0,6.0,1.0,92.0,169276.405792,13283.675264,5454.06889,3258.006066,0.155378
2,17,1,2015,1,First,1713,32.469987,20.138549,1.55,11.3,...,63.0,9.0,6.0,1.0,92.0,169276.405792,13283.675264,5454.06889,3258.006066,0.155378
3,17,1,2015,2,Second,1713,26.455588,10.647633,1.87,28.3,...,63.0,9.0,6.0,1.0,92.0,169276.405792,13283.675264,5454.06889,3258.006066,0.155378
4,17,1,2015,2,Third,1713,26.455588,10.647633,1.87,28.3,...,63.0,9.0,6.0,1.0,92.0,169276.405792,13283.675264,5454.06889,3258.006066,0.155378


In [6]:
data_list = prep_data(data, "high_crime",
                                        2, "Year", ["district", "sector", 
                                                    "Month", 
                                                    "Beat", "Watch"])

      district sector  Year Month   Watch  Beat       TMAX       TMIN  PRCP  \
29557       17      1  2018     1   First  1713  33.201426  18.884223  1.12   
29558       17      1  2018     1  Second  1713  33.201426  18.884223  1.12   
29559       17      1  2018     1   Third  1713  33.201426  18.884223  1.12   
29560       17      1  2018     2   Third  1713  39.791635  22.439919  5.47   
29561       17      1  2018     2   First  1713  39.791635  22.439919  5.47   
...        ...    ...   ...   ...     ...   ...        ...        ...   ...   
49256        3      1  2019    11  Second   312  43.761013  29.648049  1.70   
49257        3      1  2019    11   Third   312  43.761013  29.648049  1.70   
49258        3      1  2019    12  Second   312  42.647269  27.854899  1.29   
49259        3      1  2019    12   First   312  42.647269  27.854899  1.29   
49260        3      1  2019    12   Third   312  42.647269  27.854899  1.29   

       SNOW  ... count_restaurants  count_bars  cou

Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015 2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [7]:
data_list_pca = prep_data(data, "high_crime",
                                        2, "Year", 
                          ["district", "sector", 
                                                    "Month", 
                                                    "Beat", "Watch"], pca="yes!", 
                                                  columns_to_pca = ["TOTAL POPULATION", "dist_to_police",
                                                                   "dist_to_hospital", 
                                                  "count_l_stops", "count_bus_stops", 
                                               "count_metra_stops", "count_restaurants", "count_bars", 
                                               "count_daycares", "count_entertainment", "count_businesses", 
                                               "road_distance_ft"])

      district sector  Year Month   Watch  Beat       TMAX       TMIN  PRCP  \
29557       17      1  2018     1   First  1713  33.201426  18.884223  1.12   
29558       17      1  2018     1  Second  1713  33.201426  18.884223  1.12   
29559       17      1  2018     1   Third  1713  33.201426  18.884223  1.12   
29560       17      1  2018     2   Third  1713  39.791635  22.439919  5.47   
29561       17      1  2018     2   First  1713  39.791635  22.439919  5.47   
...        ...    ...   ...   ...     ...   ...        ...        ...   ...   
49256        3      1  2019    11  Second   312  43.761013  29.648049  1.70   
49257        3      1  2019    11   Third   312  43.761013  29.648049  1.70   
49258        3      1  2019    12  Second   312  42.647269  27.854899  1.29   
49259        3      1  2019    12   First   312  42.647269  27.854899  1.29   
49260        3      1  2019    12   Third   312  42.647269  27.854899  1.29   

       SNOW  ... count_restaurants  count_bars  cou

0    0.852368
1    0.852368
2    0.852368
3    0.852368
4    0.852368
Name: principal component 1, dtype: float64
(19695, 2) (19695, 322)
Working on: [2016 2017]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
0    0.85269
1    0.85269
2    0.85269
3    0.85269
4    0.85269
Name: principal component 1, dtype: float64
(19696, 2) (19696, 322)
Working on: [2015 2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
0    0.856576
1    0.856576
2    0.856576
3    0.856576
4    0.856576
Name: principal component 1, dtype: float64
(19712, 2) (19712, 322)


In [5]:
models = {
    'GaussianNB': GaussianNB(),
    'LinearSVC': LinearSVC()    
}

NBgrid = {'GaussianNB': [{'priors': None}] 
         }

SVCgrid = {'LinearSVC': [{'C': x, 'random_state': 0} for x in (0.01, 0.1, 1, 10, 100)]}

In [8]:
nb_results = average_grid_searches(GaussianNB(), 'GaussianNB', NBgrid, data_list, "high_crime", 2020)
nb_pca_results = average_grid_searches(GaussianNB(), 'GaussianNB', NBgrid, data_list_pca, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]
test year is: 2020
[2016, 2017, 2018, 2019]


In [9]:
nb_results

,parameters_2019,precision_2019,parameters_2018,precision_2018,parameters_2017,precision_2017,mean
0,{'priors': None},0.38322,{'priors': None},0.363594,{'priors': None},0.355835,0.36755


In [12]:
nb_pca_results

,parameters_2019,precision_2019,parameters_2018,precision_2018,parameters_2017,precision_2017,mean
0,{'priors': None},0.390228,{'priors': None},0.371183,{'priors': None},0.357834,0.373082


In [15]:
nb_results.to_csv("../final_data/NB.csv")

In [10]:
svc_results = average_grid_searches(LinearSVC(), 'LinearSVC', SVCgrid, data_list, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]


In [11]:
svc_pca_results = average_grid_searches(LinearSVC(), 'LinearSVC', SVCgrid, data_list_pca, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]


In [13]:
svc_results

,parameters_2019,precision_2019,parameters_2018,precision_2018,parameters_2017,precision_2017,mean
0,"{'C': 1, 'random_state': 0}",0.751169,"{'C': 1, 'random_state': 0}",1.000000,"{'C': 1, 'random_state': 0}",0.927184,0.892785
0,"{'C': 10, 'random_state': 0}",0.751169,"{'C': 10, 'random_state': 0}",1.000000,"{'C': 10, 'random_state': 0}",0.927184,0.892785
0,"{'C': 100, 'random_state': 0}",0.751169,"{'C': 100, 'random_state': 0}",1.000000,"{'C': 100, 'random_state': 0}",0.927184,0.892785
0,"{'C': 0.1, 'random_state': 0}",0.751169,"{'C': 0.1, 'random_state': 0}",1.000000,"{'C': 0.1, 'random_state': 0}",0.843496,0.864888
0,"{'C': 0.01, 'random_state': 0}",0.265156,"{'C': 0.01, 'random_state': 0}",0.282265,"{'C': 0.01, 'random_state': 0}",0.738462,0.428627


In [14]:
svc_pca_results

,parameters_2019,precision_2019,parameters_2018,precision_2018,parameters_2017,precision_2017,mean
0,"{'C': 10, 'random_state': 0}",0.912664,"{'C': 10, 'random_state': 0}",1.000000,"{'C': 10, 'random_state': 0}",0.672518,0.861727
0,"{'C': 100, 'random_state': 0}",0.912664,"{'C': 100, 'random_state': 0}",1.000000,"{'C': 100, 'random_state': 0}",0.672518,0.861727
0,"{'C': 1, 'random_state': 0}",0.912664,"{'C': 1, 'random_state': 0}",1.000000,"{'C': 1, 'random_state': 0}",0.672320,0.861661
0,"{'C': 0.1, 'random_state': 0}",0.912664,"{'C': 0.1, 'random_state': 0}",1.000000,"{'C': 0.1, 'random_state': 0}",0.247740,0.720135
0,"{'C': 0.01, 'random_state': 0}",0.910256,"{'C': 0.01, 'random_state': 0}",0.776884,"{'C': 0.01, 'random_state': 0}",0.000000,0.562380


In [16]:
svc_results.to_csv("../final_data/SVC.csv")